In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/insurance/insurance.csv


In [2]:
df = pd.read_csv("/kaggle/input/insurance/insurance.csv")

In [3]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [4]:
df['region'].unique()

array(['southwest', 'southeast', 'northwest', 'northeast'], dtype=object)

In [5]:
df['children'].unique()

array([0, 1, 3, 2, 5, 4])

In [6]:
df['smoker'].unique()

array(['yes', 'no'], dtype=object)

In [7]:
# find the number of null values in the dataframe
df.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

In [8]:
dup = df[df.duplicated()]

In [9]:
dup

,age,sex,bmi,children,smoker,region,charges
581,19,male,30.59,0,no,northwest,1639.5631


In [10]:
# replace dulpicate rows in the dataframe
df.drop_duplicates(inplace=True)

In [11]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [12]:
# label encoder

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le2 = preprocessing.LabelEncoder()
le3 = preprocessing.LabelEncoder()
le.fit(df['region'])
le2.fit(df['smoker'])
le3.fit(df['sex'])
#le.classes_
df['region'] = le.transform(df['region'])
df['smoker'] = le2.transform(df['smoker'])
df['sex'] = le3.transform(df['sex'])
df

,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,1,3,16884.92400
1,18,1,33.770,1,0,2,1725.55230
2,28,1,33.000,3,0,2,4449.46200
3,33,1,22.705,0,0,1,21984.47061
4,32,1,28.880,0,0,1,3866.85520
...,...,...,...,...,...,...,...
1333,50,1,30.970,3,0,1,10600.54830
1334,18,0,31.920,0,0,0,2205.98080
1335,18,0,36.850,0,0,2,1629.83350
1336,21,0,25.800,0,0,3,2007.94500


In [13]:
# check for outliers 
thres = 3.5
for col in ['age','bmi','charges']:
    mean = df[col].mean()
    std = df[col].std()
    print(col,'\n', df.loc[((df[col]-mean)/std) > thres])
    df=df.loc[((df[col]-mean)/std) < thres]
        

age 
 Empty DataFrame
Columns: [age, sex, bmi, children, smoker, region, charges]
Index: []
bmi 
       age  sex    bmi  children  smoker  region     charges
1047   22    1  52.58         1       1       2  44501.3982
1317   18    1  53.13         0       0       2   1163.4627
charges 
       age  sex     bmi  children  smoker  region      charges
543    54    0  47.410         0       1       2  63770.42801
577    31    0  38.095         1       1       0  58571.07448
1230   52    1  34.485         3       1       1  60021.39897
1300   45    1  30.360         0       1       2  62592.87309


In [14]:
df

,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,1,3,16884.92400
1,18,1,33.770,1,0,2,1725.55230
2,28,1,33.000,3,0,2,4449.46200
3,33,1,22.705,0,0,1,21984.47061
4,32,1,28.880,0,0,1,3866.85520
...,...,...,...,...,...,...,...
1333,50,1,30.970,3,0,1,10600.54830
1334,18,0,31.920,0,0,0,2205.98080
1335,18,0,36.850,0,0,2,1629.83350
1336,21,0,25.800,0,0,3,2007.94500


In [15]:
# Run mulitvariate linear regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
#check relationship between variables
#plt.scatter(df['age'], df['bmi'], color='red')
#plt.scatter(df['age'], df['charges'], color='blue')
#plt.scatter(df['bmi'], df['charges'], color='blue')

y = df['charges']
x = df.drop(columns = 'charges')
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
regr = LinearRegression()
regr.fit(x_train, y_train)

LinearRegression()

In [16]:
print("Coefficients: ", regr.coef_)
print("Intercept: ", regr.intercept_)
print("Score: ", regr.score(x_test, y_test))

Coefficients:  [  256.73475309  -184.38517527   319.9929979    527.93908169
 22795.29459478  -313.33601704]
Intercept:  -11453.427518771907
Score:  0.7991085917708144


In [17]:
# standardization of the dataframe to improve accuracy
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
y_train_scaled = scaler.fit_transform(pd.DataFrame(y_train))
regr_scaled = LinearRegression()
regr_scaled.fit(x_train_scaled, y_train_scaled)

LinearRegression()

In [18]:
x_test_scaled = scaler.fit_transform(x_test)
y_test_scaled = scaler.fit_transform(pd.DataFrame(y_test))
print("Coefficients: ", regr_scaled.coef_)
print("Intercept: ", regr_scaled.intercept_)
print("Score: ", regr_scaled.score(x_test_scaled, y_test_scaled))

Coefficients:  [[ 0.31149443 -0.00797722  0.16647603  0.05495826  0.7822671  -0.02962258]]
Intercept:  [-2.800696e-16]
Score:  0.8027805646063212


In [19]:
#F_regression
from sklearn.feature_selection import f_regression

f_stat, p_val = f_regression(x_train, y_train)
p_val.round(3)

#P_values are = array([0.   , 0.053, 0.   , 0.011, 0.   , 0.963])
#the second and fourth variable has p_values greater than 0.05 but we can choose to remove the fourth variable

array([0.   , 0.053, 0.   , 0.011, 0.   , 0.963])

In [20]:
#removing the 4th variable ('region') after f_regression
df1 = df.drop(columns = 'region')

In [21]:
#linear regression with feature scaling after removing the 4th variable ('region')
y1 = df1['charges']
x1 = df1.drop(columns = 'charges')
x_train1, x_test1, y_train1, y_test1 = train_test_split(x1, y1, test_size=0.2, random_state=42)
x_train1_scaled = scaler.fit_transform(x_train1)
y_train1_scaled = scaler.fit_transform(pd.DataFrame(y_train1))
regr1 = LinearRegression()
regr1.fit(x_train1_scaled, y_train1_scaled)
x_test1_scaled = scaler.fit_transform(x_test1)
y_test1_scaled = scaler.fit_transform(pd.DataFrame(y_test1))
print("Coefficients: ", regr1.coef_)
print("Intercept: ", regr1.intercept_)
print("Score: ", regr1.score(x_test1_scaled, y_test1_scaled))

#Removing the 4th variable didnot improve the accuracy. Hence "regr_scaled" is chosen as the final model

Coefficients:  [[ 0.31214095 -0.00756935  0.16151632  0.05495298  0.78208409]]
Intercept:  [-2.81195031e-16]
Score:  0.8015224821248985
